***DATA ANALYSIS***

In [85]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os #file I/O
from sklearn.preprocessing import StandardScaler
import seaborn as sns

import os

%matplotlib inline

import matplotlib.pyplot as plt #used for plotting and analyzing data

#used for image display
import PIL 
from PIL import Image

# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.python.keras.models import Sequential
from tensorflow.keras import layers

import matplotlib.pyplot as plt
from sklearn.metrics import auc,roc_curve
import seaborn as sns
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import PIL 
from PIL import Image

In [86]:
import os
print(os.listdir("../input"))

['skin-cancer9-classesisic']


In [87]:
path='../input/'

In [88]:
lesion_type_dict = {
    'NV': 'Melanocytic nevi',
    'MEL': 'Melanoma',
    'BKL': 'Benign keratosis ',
    'BCC': 'Basal cell carcinoma',
    'AK': 'Actinic keratoses',
    'VASC': 'Vascular lesions',
    'DF': 'Dermatofibroma',
    'SCC' : 'Squamous cell carcinoma'
} 

***THE PREPROCESSING OF SKIN IMAGES***

In [89]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import textwrap as tw
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.applications.resnet import preprocess_input
# from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.python.keras.models import Sequential
from tensorflow.keras import layers


In [90]:
# # print all columns
# pd.set_option('display.max_columns', None)

# # inhibit graphics card runs out of memory
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)


In [91]:
def load_data(path: str):
    dir = Path(path)
    # list of all filepathes
    filepaths = list(dir.glob(r'**/*.jpg'))
    # list of labels extracted from last foldername of filepath
    labels = list(map(lambda l: os.path.split(os.path.split(l)[0])[1], filepaths))
    # series of string filepathes
    filepaths = pd.Series(filepaths, name='FilePaths').astype(str)
    # series of string labels
    labels = pd.Series(labels, name='Labels').astype(str)
    # merge series to dataframe df
    df = pd.merge(filepaths, labels, right_index=True, left_index=True)
    # Resampling complete rows and reset the index
    return df.sample(frac=1).reset_index(drop=True)
df = load_data('../input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train')

In [92]:
# total categires 
len(os.listdir('../input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train'))

9

In [93]:
os.listdir('../input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train')

['pigmented benign keratosis',
 'melanoma',
 'vascular lesion',
 'actinic keratosis',
 'squamous cell carcinoma',
 'basal cell carcinoma',
 'seborrheic keratosis',
 'dermatofibroma',
 'nevus']

In [94]:
list_diseases = os.listdir('../input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train')

In [95]:
results2 = []
for disease in list_diseases:
    dies_name_count = {}
    count_disease = len(os.listdir(f'../input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train'))
    dies_name_count['disease'] = disease
    dies_name_count['count_images'] = count_disease
    results2.append(dies_name_count)

In [96]:
results = pd.DataFrame(results2)
results

,disease,count_images
0,pigmented benign keratosis,9
1,melanoma,9
2,vascular lesion,9
3,actinic keratosis,9
4,squamous cell carcinoma,9
5,basal cell carcinoma,9
6,seborrheic keratosis,9
7,dermatofibroma,9
8,nevus,9


In [97]:
df.head(25)

,FilePaths,Labels
0,../input/skin-cancer9-classesisic/Skin cancer ...,basal cell carcinoma
1,../input/skin-cancer9-classesisic/Skin cancer ...,basal cell carcinoma
2,../input/skin-cancer9-classesisic/Skin cancer ...,nevus
3,../input/skin-cancer9-classesisic/Skin cancer ...,nevus
4,../input/skin-cancer9-classesisic/Skin cancer ...,basal cell carcinoma
5,../input/skin-cancer9-classesisic/Skin cancer ...,seborrheic keratosis
6,../input/skin-cancer9-classesisic/Skin cancer ...,melanoma
7,../input/skin-cancer9-classesisic/Skin cancer ...,vascular lesion
8,../input/skin-cancer9-classesisic/Skin cancer ...,vascular lesion
9,../input/skin-cancer9-classesisic/Skin cancer ...,vascular lesion


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2239 entries, 0 to 2238
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   FilePaths  2239 non-null   object
 1   Labels     2239 non-null   object
dtypes: object(2)
memory usage: 35.1+ KB


In [99]:
# # ordered count of rows per unique label
# labels_count = df['Labels'].value_counts(ascending=True)

# f = plt.figure(figsize=(15, 6))
# s = sns.barplot(labels_count.index,labels_count.values)
# sns.despine()
# s.set_xticklabels(s.get_xticklabels(), rotation = 30)

In [100]:
# def plot_images_per_label(df, label, cols: int, size: tuple):
#     fig, axs = plt.subplots(nrows=1, ncols=cols, figsize=size)

#     cntMax = cols
#     cntCur = 0
#     for index, row in df.iterrows():
#         if(row['Labels'] == label and cntCur < cntMax):
#             axs[cntCur].imshow(plt.imread(df.FilePaths[index]))
#             axs[cntCur].set_title(df.Labels[index])

#             cntCur += 1
#         else:
#             if(cntCur >= cntMax):
#                 break
    
#     plt.tight_layout()
#     plt.show()


# # unique labels
# labels = sorted(df['Labels'].unique())
# # loop through labels
# for label in labels:
#     plot_images_per_label(df, label, 3, (12,9))

In [101]:
# stratified train and val (20%) datasets
X_train, X_val = train_test_split(df, test_size=0.2, stratify=df['Labels'], random_state=1)

print('Train Data: ', X_train.shape)
print('Val Data: ', X_val.shape)

Train Data:  (1791, 2)
Val Data:  (448, 2)


In [102]:
# # number of samples/images per iteration
# BATCH_SIZE = 32
# # input image size
# IMG_SIZE = (224, 224)
# # count of epchos
# EPOCHS = 2

# # image preprocessing
# img_data_gen = ImageDataGenerator(shear_range=0.2,
#                                   zoom_range=0.2,
#                                   horizontal_flip=True,
#                                   preprocessing_function=preprocess_input)


# img_data_gen_test = ImageDataGenerator()



# X_train = img_data_gen.flow_from_dataframe(dataframe=X_train,
#                                            x_col='FilePaths',
#                                            y_col='Labels',
#                                            target_size=IMG_SIZE,
#                                            color_mode='rgb',
#                                            class_mode='categorical',
#                                            batch_size=BATCH_SIZE,
#                                            seed=1)


# X_val = img_data_gen_test.flow_from_dataframe(dataframe=X_val,
#                                          x_col='FilePaths',
#                                          y_col='Labels',
#                                          target_size=IMG_SIZE,
#                                          color_mode='rgb',
#                                          class_mode='categorical',
#                                          batch_size=BATCH_SIZE,
#                                          seed=1)

In [103]:
import random
batch_size = 32
img_height = 180
img_width = 180
rnd_seed = 123
random.seed(rnd_seed)

In [104]:
data_dir_train = '../input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train'
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2239 files belonging to 9 classes.
Using 1792 files for training.


In [105]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2239 files belonging to 9 classes.
Using 447 files for validation.


In [106]:
# fit, ax = plt.subplots(nrows=3, ncols=3, figsize=(12,27))

# for i, a in enumerate(ax.flat):
#     img, label = X_train.next()
#     a.imshow(img[0],)
#     a.set_title(label[0])

# plt.tight_layout()
# plt.show()

In [107]:
num_classes = 9
model = Sequential([layers.experimental.preprocessing.Rescaling \
                    (1.0/255,input_shape=(img_height,img_width,3))])

model.add(Conv2D(32, 3,padding="same",activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(64, 3,padding="same",activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(128, 3,padding="same",activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(256, 3,padding="same",activation='relu'))
model.add(MaxPool2D())

model.add(Conv2D(512, 3,padding="same",activation='relu'))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dense(units=num_classes, activation= 'softmax'))

In [108]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [ ]:
#compiling the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
# stop training when accuracy has stopped improving 
# cb = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)
# hst = model.fit(X_train, validation_data=X_val, epochs=EPOCHS, callbacks=cb)
# train model 
hst = model.fit(X_train, validation_data=X_val, epochs=EPOCHS)

In [ ]:
# model.save_weights('model',save_format='tf')

In [ ]:
# model.load_weights('model')

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
acc = hst.history['acc']
val_acc = hst.history['val_acc']
loss = hst.history['loss']
val_loss = hst.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs,acc,'r', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'b', label = 'validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc= 0)
plt.figure()


In [ ]:
X_test = load_data('../input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test')

In [ ]:
X_test.head(25)

In [ ]:
print('Test Data: ', X_test.shape)

# ordered count of rows per unique label
X_test['Labels'].value_counts(ascending=True)

In [ ]:
# image preprocessing
X_test = img_data_gen.flow_from_dataframe(dataframe=X_test,
                                          x_col='FilePaths',
                                          y_col='Labels',
                                          target_size=IMG_SIZE,
                                          color_mode='rgb',
                                          class_mode='categorical',
                                          batch_size=BATCH_SIZE,
                                          shuffle=False, # necessary fpr confusion matrix
                                          seed=1)

In [ ]:
import PIL.Image as Image

In [ ]:
import PIL.Image as Image

In [ ]:
image_path = '/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test/vascular lesion/ISIC_0024370.jpg'

In [ ]:
IMAGE_SHAPE = (224, 224)

In [ ]:
image = Image.open(image_path).resize(IMAGE_SHAPE)
image

In [ ]:
image = np.array(image)/255.0
image.shape

In [ ]:
result = model.predict(image[np.newaxis, ...])
result


# array([[0.0000000e+00, 2.8053434e-35, 0.0000000e+00, 3.6857502e-37,
#         0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 3.4883269e-26,
#         0.0000000e+00]], dtype=float32)

In [ ]:
# predicted_label_index = np.argmax(result)
# predicted_label_index
np.argmax(result)

In [ ]:
# image

In [ ]:
res = model.evaluate(X_test)

In [ ]:
# accuracy
print(f'Train Accuracy: {hst.history["acc"][-1:][0] * 100:.2f}')
print(f'Val Accuracy: {hst.history["val_acc"][-1:][0] * 100:.2f}')
print(f'Test Accuracy: {res[1] * 100:.2f}')
# loss
print(f'Train Loss: {hst.history["loss"][-1:][0] * 100:.2f}')
print(f'Val Loss: {hst.history["val_loss"][-1:][0] * 100:.2f}')
print(f'Test Loss: {res[0] * 100:.2f}')

In [ ]:
# predicted labels
Y_pred = model.predict(X_test)
print("Y_pred", Y_pred.shape)
# rounded labels
y_pred = np.argmax(Y_pred, axis=1)
print("y_pred", y_pred.size)

In [ ]:
# true labels
y_true = X_test.classes
print("y_pred", len(y_pred))
# label classes
class_labels = list(X_test.class_indices.keys())
print("labels", len(class_labels))

In [ ]:
# compare with true labels
cfm = confusion_matrix(y_pred, y_true, normalize='true')

# plot size
fig, ax = plt.subplots(figsize=(15,15))
# print confusion matrix
s = sns.heatmap(cfm,
               annot=True,
               cmap=['#ff0001', '#09AA11'],
               center=0.8,
               fmt='.1%',
               linewidths=.5,
               cbar_kws={'format': FuncFormatter(lambda x, pos: '{:.0%}'.format(x))}, #'label': 'Percentage' 
               linecolor='Black',
               ax=ax)
# set labels
s.set(xlabel='Predict', ylabel='True')
s.set(title='Confusion Matrix')
s.set_yticklabels([tw.fill(e, 10) for e in class_labels])
s.set_xticklabels([tw.fill(e, 10) for e in class_labels])